In [1]:
from sklearn import datasets
import pandas as pd
from int_met import *

In [2]:
x = datasets.load_breast_cancer()
#x=pd.read_csv('Breast cancer.csv')  

In [3]:
data = x.data
feature_names = x.feature_names
y = x.target

In [4]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
model = RandomForestClassifier()
#model=DecisionTreeClassifier()


In [5]:
train_data = data[1:300,:]
train_labels=y[1:300]
test_data = data[301:505,:]
test_labels=y[301:505]

In [6]:
model = model.fit(train_data,y=train_labels)

In [7]:
from sklearn.metrics import f1_score
y_pred = model.predict(test_data)
error = f1_score(test_labels,y_pred) 
error
y_pred

array([1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 0, 1])

In [8]:
#necessary imports
from pyBreakDown.explainer import Explainer
from pyBreakDown.explanation import Explanation

In [9]:
exp = Explainer(clf=model, data=train_data, colnames=feature_names)

In [10]:
def run_PyBreakDown(model,data,observations,colnames):
    explanations=[]
    y=[]
    from pyBreakDown.explainer import Explainer
    from pyBreakDown.explanation import Explanation
    from math import exp as e
    exp = Explainer(clf=model, data=data, colnames=feature_names)
    if observations.ndim==1:
        explanation = exp.explain(observation=observations,direction="up")
        pred=explanation._final_prediction
        pred=round((e(pred)/(1+e(pred))))
        y.append(pred)
        explanations.append(explanation)
    else:
        for i in range(0,len(observations)):
            observation = observations[i]
            explanation = exp.explain(observation=observation,direction="up")
            pred=explanation._final_prediction
            pred=round((e(pred)/(1+e(pred))))
            y.append(pred)
            explanations.append(explanation)
    return explanations,y

In [11]:
def get_Coefs(explanations):
    coefs=[]
    for explanation in explanations:
        coef=[]
        for feature in explanation._attributes:
            coef.append(feature.contribution)
        coefs.append(coef)
    return coefs

In [12]:
explanations1,y1 = run_PyBreakDown(model=model,data=train_data,observations=data[567:569,:],colnames=feature_names)
explanations2,y2 = run_PyBreakDown(model=model,data=train_data,observations=data[567:569,:],colnames=feature_names)

In [13]:
coefs1 = get_Coefs(explanations1)
coefs2 = get_Coefs(explanations2)
y_test = y[567:569]
X_test = data[567:569,:]

In [14]:
calc_identity(coefs1,coefs2)

(0.0, 2, 2)

In [15]:
calc_stability(coefs1, y_test)

/home/abdul/Desktop/interp/XAI-SPRING2020/MyBreakDown/int_met.py:51: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  ct.fit(exp)


(0, 2)

In [16]:
calc_separability(X_test, coefs1)

(2, 0, 100.0)

In [26]:
exp=coefs1
exp = np.asarray(exp)
for coef in coefs1:
    print(len(coef)-np.count_nonzero(coef))

26
22


In [27]:
def Compactness(exps):
    import pandas as pd
    import numpy as np
    exps = np.asarray(exps)
    scores=[]
    for exp in exps:
        scores.append((len(exp)-np.count_nonzero(exp))/len(exp))
    return (np.average(scores))

In [28]:
Compactness(coefs1)

0.7741935483870968